In [1]:
# Import requests library
import requests
import pandas as pd
import sqlite3

***Get Available Crimes***

*Parse Crimes Class*

In [8]:
class extract_transform_load_data:
    
    def get_forces(self):
        # Get all available forces
        forces = requests.get('https://data.police.uk/api/forces')
        # Check if the request was successful (status code 200)
        if forces.status_code == 200:
            # Parse the JSON response
            forces_data = forces.json()
        
            # Extract 'id' from each item in the response and save it into a list
            force_id_list = [force.get('id') for force in forces_data]
            return force_id_list
        else:
            return None

    
    def get_crimes(self, year, month, force_id):
        crimes = requests.get(f'https://data.police.uk/api/crimes-no-location?category=all-crime&force={force_id}&date={year}-{month}')
        if crimes.status_code == 404:
            print(f'Error: No data for force_id {force_id}')
            return pd.DataFrame()  # Return an empty DataFrame
        else:
            crimes_data = crimes.json()
            # Extract specific information from the JSON data
            relevant_info = []

            for crime in crimes_data:
                # Check if 'category' is present and not null
                if crime.get('outcome_status', {}):
                    relevant_info.append({
                        'id': crime.get('id'),
                        'city': force_id,
                        'category': crime.get('category'),
                        'status': crime.get('outcome_status', {}).get('category', ''),
                        'date': crime.get('outcome_status', {}).get('date', ''),
                        # Add more fields as needed
                    })

            # Create a DataFrame from the extracted information
            df = pd.DataFrame(relevant_info)
            return df
    
    def process_data(self, year, month):
        forces_list = self.get_forces()
        all_dfs = []  # To store individual DataFrames

        # Iterate through force_ids and fetch data for each force
        for force_id in forces_list:
            result_df = self.get_crimes(year, month, force_id)

            if result_df is not None:
                # Add the DataFrame to the list
                all_dfs.append(result_df)

        # Concatenate all DataFrames into a single DataFrame
        final_df = pd.concat(all_dfs, ignore_index=True)
        return final_df

    
    def load_repo(self, year, month):
        # Connect to a database (or create it if it doesn't exist)
        connection = sqlite3.connect(f'crime_data_{year}_{month}.db')

        # Create a cursor
        cursor = connection.cursor()

        # Execute a SQL query to create a table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS crimes (
                id INTEGER PRIMARY KEY,
                city TEXT,
                category TEXT,
                status TEXT,
                date DATE
            )
        ''')

        # Process data to get the DataFrame
        aggregated_data = self.process_data(year, month)

        # Check if the DataFrame is not empty before saving it to the database
        if not aggregated_data.empty:
            # Save the DataFrame to the SQLite database
            aggregated_data.to_sql('crimes', connection, if_exists='replace', index=False)

            # Commit the changes and close the connection
            connection.commit()
            connection.close()
            print("Data loaded to the database successfully.")
        else:
            print("No data to load to the database.")
        
        

In [10]:
etl_session = extract_transform_load_data()
etl_session.load_repo(2023, 8)

Data loaded to the database successfully.
